In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
## Parameters
w, b, q = 10, 20, 1/10

## Target function
f = lambda x,y: b*(1-np.exp(-1/2*w*(x**2 + y**2))) + 1/2*q*(y - x**3)**2

## Gradient of the function
def d_f_x(params):
    x, y = params
    return b*w*np.exp(-1/2*w*(x**2 + y**2))*x - 3*q*(y - x**3)*x**2
def d_f_y(params):
    x, y = params
    return b*w*np.exp(-1/2*w*(x**2 + y**2))*y + q*(y-x**3)

g_f = lambda params: np.array([d_f_x(params),d_f_y(params)])



In [ ]:
## Gradient methods. Adapted from mlreview_notebooks.

#Mean-gradient based methods
def gd(grad, init, n_epochs=1000, eta=10**-2, noise_strength=0):
    #This is a simple optimizer
    params=np.array(init)
    param_traj=np.zeros([n_epochs+1,2])
    param_traj[0,]=init
    v=0;
    for j in range(n_epochs):
        noise=noise_strength*np.random.randn(params.size)
        v=eta*(np.array(grad(params))+noise)
        params=params-v
        param_traj[j+1,]=params
    return param_traj


def gd_with_mom(grad, init, n_epochs=5000, eta=10**-2, gamma=0.9,noise_strength=0):
    params=np.array(init)
    param_traj=np.zeros([n_epochs+1,2])
    param_traj[0,]=init
    v=0
    for j in range(n_epochs):
        noise=noise_strength*np.random.randn(params.size)
        v=gamma*v+eta*(np.array(grad(params))+noise)
        params=params-v
        param_traj[j+1,]=params
    return param_traj

def NAG(grad, init, n_epochs=5000, eta=10**-2, gamma=0.9,noise_strength=0):
    params=np.array(init)
    param_traj=np.zeros([n_epochs+1,2])
    param_traj[0,]=init
    v=0
    for j in range(n_epochs):
        noise=noise_strength*np.random.randn(params.size)
        params_nesterov=params-gamma*v
        v=gamma*v+eta*(np.array(grad(params_nesterov))+noise)
        params=params-v
        param_traj[j+1,]=params
    return param_traj

#Methods that exploit first and second moments of gradient: RMS-PROP and ADAMS

def rms_prop(grad, init, n_epochs=5000, eta=10**-2, beta=0.9,epsilon=10**-8,noise_strength=0):
    params=np.array(init)
    param_traj=np.zeros([n_epochs+1,2])
    param_traj[0,]=init#Import relevant packages
    grad_sq=0;
    for j in range(n_epochs):
        noise=noise_strength*np.random.randn(params.size)
        g=np.array(grad(params))+noise
        grad_sq=beta*grad_sq+(1-beta)*g*g
        v=eta*np.divide(g,np.sqrt(grad_sq+epsilon))
        params= params-v
        param_traj[j+1,]=params
    return param_traj
                        
                        
def adams(grad, init, n_epochs=5000, eta=10**-2, gamma=0.9, beta=0.99,epsilon=10**-8,noise_strength=0):
    params=np.array(init)
    param_traj=np.zeros([n_epochs+1,2])
    param_traj[0,]=init
    v=0;
    grad_sq=0;
    for j in range(n_epochs):
        noise=noise_strength*np.random.randn(params.size)
        g=np.array(grad(params))+noise
        v=gamma*v+(1-gamma)*g
        grad_sq=beta*grad_sq+(1-beta)*g*g
        v_hat=v/(1-gamma)
        grad_sq_hat=grad_sq/(1-beta)
        params=params-eta*np.divide(v_hat,np.sqrt(grad_sq_hat+epsilon))
        param_traj[j+1,]=params
    return param_traj


In [ ]:
## Building initial positions grid:
# Couldn't make this work the way I intended
# xv, yv = np.meshgrid(np.linspace(-3,3),np.linspace(-3,3),sparse=True)
## I don't love this approach, too programming 101
init_points = []
x = np.linspace(-3,3)
y = np.linspace(-3,3)
for i in x:
    for j in y:
        init_points.append([i,j])
# init_points is now the [-3,3]x[-3,3] grid

In [ ]:
## 
def get_step_avg(method,f,grad_f,init_points, n_epochs = 1000):
    results = []
    for init_pair in init_points:
        # too reliant on the brute force grid
        results.append(method(grad_f,init_pair,n_epochs=n_epochs))
    tray = np.array(results)
    step_avg = np.average(f(tray[:,:,0],tray[:,:,1]), axis = 0)
    return step_avg

In [ ]:
h1 = get_step_avg(gd,f,g_f,init_points, n_epochs = 100)
h2 = get_step_avg(gd_with_mom,f,g_f,init_points, n_epochs = 100)
h3 = get_step_avg(NAG,f,g_f,init_points, n_epochs = 100)
h4 = get_step_avg(rms_prop,f,g_f,init_points, n_epochs = 100)
h5 = get_step_avg(adams,f,g_f,init_points, n_epochs = 100)

In [ ]:
t = np.linspace(0,len(h1),num=len(h1))

plt.rcParams["figure.figsize"] = (12,8)
plt.plot(t,h1,label='gd')
plt.plot(t,h2,label='gd_with_mom')
plt.plot(t,h3,label='NAG')
plt.plot(t,h4,label='rms_prop')
plt.plot(t,h5,label='adams')
plt.ylabel('Average',fontsize=17)  
plt.xlabel('Epoch',fontsize=17)
plt.legend()
plt.show()